In [6]:
import os
from os.path import join, splitext
import numpy as np
from tqdm.notebook import tqdm

In [4]:
data_dir = '../data/ALLEN/static_grating_orientation'
neural_dir = 'neural'
target_dir = 'target'

In [5]:
neural_filenames = sorted(os.listdir(join(data_dir, neural_dir)))
n_neuron = 0
for neural_filename in neural_filenames:
    neural_arr = np.load(join(data_dir, join(neural_dir, neural_filename)))
    n_neuron += neural_arr.shape[0]
    n_timestamps = neural_arr.shape[1]

In [7]:
# Initialize a big total neural arr
cnt_per_target = 800
filename = 'VISp_static_grating_neural.npy'
total_neural_arr = np.memmap(join(data_dir, filename), dtype='float64', mode='w+', shape=(n_neuron, n_timestamps, cnt_per_target*6)) # 6 unique targets
curr_total_n_neuron = 0
# Loop into each smaller neural arr from each experiment
for neural_filename in tqdm(neural_filenames):
    # Get target filename
    neural_arr = np.load(join(data_dir, join(neural_dir, neural_filename)))
    target_filename = splitext(neural_filename)[0].split('_')
    target_filename[-2] = 'orientations'
    target_filename = '_'.join(target_filename) + '.npy'

    # Get a sorted neural arr in terms of unique targets
    target_arr = np.load(join(data_dir, join(target_dir, target_filename)))
    uniq_targets = np.unique(target_arr)
    sorted_neural_idx = []
    for target in uniq_targets:
        sorted_neural_idx.extend(np.where(target_arr == target)[0][:cnt_per_target])
    sorted_neural_arr = neural_arr[:, :, sorted_neural_idx]

    # Save this neural arr into the total neural arr
    curr_n_neuron = sorted_neural_arr.shape[0]
    total_neural_arr[curr_total_n_neuron:curr_total_n_neuron+curr_n_neuron, :, :] = sorted_neural_arr
    curr_total_n_neuron += curr_n_neuron

total_neural_arr.flush()

  0%|          | 0/31 [00:00<?, ?it/s]

In [8]:
total_neural_arr.shape

(1963, 250, 4800)

In [9]:
for i in tqdm(range(total_neural_arr.shape[-1])):
    np.save(join(data_dir, f'neural_per_exp/VISp_static_grating_neural_trial_{i}.npy'), total_neural_arr[:, :, i])

  0%|          | 0/4800 [00:00<?, ?it/s]

In [10]:
np.load(join(data_dir, f'neural_per_exp/VISp_static_grating_neural_trial_555.npy')).shape

(1963, 250)